# Thunor Core Tutorial

**Thunor** (pronounced THOO-nor) is a free software platform for managing,
visualizing, and analyzing high throughput cell proliferation data,
which measure the dose-dependent response of cells to one or more drug(s).

This repository, [Thunor Core](https://github.com/alubbock/thunor), is a
Python package which can be used for standalone analysis or integration
into computational pipelines.

A web interface is also available, called [Thunor Web](https://github.com/alubbock/thunor-web).
Thunor Web has a [comprehensive manual](https://docs.thunor.net), which goes
into further detail about the curve fitting methods, types of plots
available and other information you may find relevant.

Please see the [Thunor website](https://www.thunor.net) for additional resources,
and a link to our chat room, where you can ask questions about Thunor.

## Start Jupyter Notebook

Run jupyter notebook with the following argument:

    jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
    
The data rate limit needs to be increased or `init_notebook_mode()` throws an error.
This is a [plotly requirement](https://community.plot.ly/t/tips-for-using-plotly-with-jupyter-notebook-5-0-the-latest-version/4156).

## Check Thunor Core is available

In [ ]:
# If the import doesn't work, uncomment the following two lines, or "pip install thunor"
import os, sys
sys.path.insert(0, os.path.abspath('../'))

import thunor

## Load a file

First, specify a file to load. Here, we use an example dataset from the thunor package itself.

In [ ]:
hts007_file = '../thunor/testdata/hts007.h5'

Load the file using `read_hdf` (for HDF5 files), `read_vanderbilt_hts` (for CSV files), or another appropriate reader.

In [ ]:
from thunor.io import read_hdf
hts007 = read_hdf(hts007_file)

We'll just use a subset of the drugs, to make the plots manageable.

In [ ]:
hts007r = hts007.filter(drugs=['cediranib', 'everolimus', 'paclitaxel'])

In [ ]:
hts007r.drugs

In [ ]:
hts007r.cell_lines

## Calculate DIP rates and parameters

These two operations can be done in two lines of code (plus imports). Note that you may see `RuntimeWarning` messages, which indicates that some dose response curves were not able to be fitted. This can happen if the cells do not stop proliferating in response to drug, the response is not closely approximated by a log-logistic curve, or the data are very noisy.

In [ ]:
from thunor.dip import dip_rates
from thunor.curve_fit import fit_params

ctrl_dip_data, expt_dip_data = dip_rates(hts007r)
fp = fit_params(ctrl_dip_data, expt_dip_data)

## Setting up plots

Each of the `plot_X` functions returns a plotly `Figure` object which can be visualised in a number of ways. Here, we use the offline `iplot` function, which generates a plot for use with Jupyter notebook. We could also generate plots using the `plot` function in standalone HTML files. See the [plotly documentation](https://plot.ly/python/offline/) for more information on the latter approach. 

In [ ]:
from thunor.plots import plot_drc, plot_drc_params, plot_time_course, plot_ctrl_dip_by_plate, plot_plate_map

## Plot Types

### Plot DIP rate curves

In [ ]:
plot_drc(fp)

### Plot DIP parameters

In [ ]:
plot_drc_params(fp, 'auc')

### Filtering fit params

The `fp` object is a pandas data frame, so we can filter it before plotting. Some examples:

In [ ]:
fit_params_bt20_pac = fp[fp.index.isin(['BT20'], level='cell_line') & \
                         fp.index.isin(['paclitaxel'], level='drug')]

plot_drc(fit_params_bt20_pac)

### Plot time course

Time course plot for paclitaxel on BT20 cells:

In [ ]:
plot_time_course(
    hts007.filter(drugs=['paclitaxel'], cell_lines=['BT20'])
)

### Quality control check: plot DIP rate ranges by cell line and plate (box plot)

In [ ]:
plot_ctrl_dip_by_plate(ctrl_dip_data)

### Quality control check: plot DIP rate as a plate heat map

In [ ]:
plate_data = hts007.plate('HTS007_149-28A', include_dip_rates=True)

In [ ]:
plot_plate_map(plate_data, color_by='dip_rates')